In [1]:
import pandas
import geopandas
%matplotlib inline

In [2]:
fname = '/g/data/v10/projects/gaip-scripts/harvest-collection/collection-completeness.h5'

Open the file see what datasets have been recored.

In [3]:
store = pandas.HDFStore(fname, 'r')
store.keys()

['/oth_and_children_products', '/sys_products']

Read the ORTHO and SYS records

In [4]:
oth_df = store['/oth_and_children_products']
sys_df = store['/sys_products']

Merge the two tables

In [5]:
df = pandas.concat([oth_df, sys_df], keys=['oth', 'sys'])

The counts recorded for L1_L1T, L1_L1Gt, L0_success are recorded as a pass level. So the values will actually
be duplicated where records/scenes have come from the same pass.

So what we need to do is find the children products, and calculate a sum. Then remove any records with a 
duplicate `pass_name`. This should yield a count of products, including any children products, for a given pass.

In [6]:
def merge_children(dataframe):
    cols = ['pass_name', 'nbar_exists', 'nbart_exists', 'pq_exists']
    children = pandas.DataFrame(columns=cols)
    cols.remove('pass_name')
    
    # for each scene with the same pass name, group and sum the records
    groups = dataframe.groupby('pass_name')
    for name, group in groups:
        res = group[cols].sum()
        res['pass_name'] = name
        children = children.append(res, ignore_index=True)
        
    children.rename(columns={'nbar_exists': 'nbar',
                             'nbart_exists': 'nbart',
                             'pq_exists': 'pq'}, inplace=True)
    
    # merge the input dataframe and the summed child products, and drop any duplicate passes
    merged = pandas.merge(dataframe, children, on=['pass_name'])
    result = merged.drop_duplicates('pass_name')
    
    return result

In [7]:
merged = merge_children(df)

Define the columns that'll be used for reporting.

In [8]:
cols = ['sensor',
        'date',
        'L0_fail',
        'L0_success',
        'L1_fail',
        'L1_success',
        'L1_L1G',
        'L1_L1Gt',
        'L1_L1T',
        'nbar',
        'nbart',
        'pq']
result = merged[cols]
result.set_index('date', inplace=True)

In [9]:
result.head(5)

,sensor,L0_fail,L0_success,L1_fail,L1_success,L1_L1G,L1_L1Gt,L1_L1T,nbar,nbart,pq
date,,,,,,,,,,,
1991-05-21,LS5,0.0,25.0,1.0,23.0,8.0,0.0,15.0,12.0,12.0,12.0
1991-05-31,LS5,0.0,25.0,1.0,23.0,10.0,0.0,13.0,13.0,13.0,13.0
1991-05-02,LS5,0.0,24.0,1.0,23.0,8.0,0.0,15.0,15.0,15.0,15.0
1991-05-05,LS5,0.0,26.0,1.0,24.0,6.0,0.0,18.0,15.0,15.0,15.0
1991-05-08,LS5,0.0,24.0,1.0,22.0,4.0,0.0,18.0,18.0,18.0,18.0


Output the results on a per-sensor basis, with aggregate summation based on the Month, to the Excel file format.

Excel format requires xlwt which is currently not available within the `agdc-py2-prod` module.

So we'll output to csv.

In [10]:
def collection_stats(dataframe, out_fname_fmt):
    groups = dataframe.groupby('sensor')
    for sensor, group in groups:
        outdf = group.resample('M').apply(sum)

        exp = "L0_completeness = L0_success / (L0_success + L0_fail) * 100"
        outdf.eval(exp, inplace=True)
        exp = "L1_completeness = L1_success / (L1_success + L1_fail) * 100"
        outdf.eval(exp, inplace=True)
        exp = "nbar_completeness = nbar / (L1_L1T + L1_L1Gt) * 100"
        outdf.eval(exp, inplace=True)
        exp = "nbart_completeness = nbart / (L1_L1T + L1_L1Gt) * 100"
        outdf.eval(exp, inplace=True)
        exp = "pq_completeness = pq / (L1_L1T + L1_L1Gt) * 100"
        outdf.eval(exp, inplace=True)
        exp = "pq_completeness_relative = pq / nbar * 100"
        outdf.eval(exp, inplace=True)

        out_fname = out_fname_fmt.format(sensor)
        outdf.to_csv(out_fname)
        # outdf.to_excel(out_fname)

In [11]:
# out_fname_fmt = '/g/data/v10/projects/gaip-scripts/harvest-collection/{}.xls'
out_fname_fmt = '/g/data/v10/projects/gaip-scripts/harvest-collection/{}.csv'
collection_stats(result, out_fname_fmt)

We can also filter the scenes to report only on those that are within GA's nominal path/row schema.

In [12]:
def subset_to_nominal_path_row(dataframe):
    """
    Given a `pandas.DataFrame` created from a trawling of lpgs_out.xml files,
    subset the list of scenes to GA's nominal path/row processing selection.
    
    :param dataframe:
        A `pandas.DataFrame`.
        
    :return:
        A `geopandas.GeoDataFrame`.
    """
    nominal_fname = '/g/data/v10/eoancillarydata/ga-nominal-scenes/ADGC_v2_Area_of_Interest.shp'
    gdf = geopandas.read_file(nominal_fname)
    gdf.rename(columns={'PATH': 'path', 'ROW': 'row'}, inplace=True)
    
    df = pandas.merge(dataframe, gdf, on=['path', 'row'])
    
    return df

In [13]:
oth_df = store['/oth_and_children_products']
sys_df = store['/sys_products']
df = pandas.concat([oth_df, sys_df], keys=['oth', 'sys'])

In [14]:
nominal = subset_to_nominal_path_row(df)

In [15]:
print "No. scenes in whole collection: {}".format(df.shape[0])
print "No. of scenes within Nominal processing: {}".format(nominal.shape[0])

No. scenes in whole collection: 456007
No. of scenes within Nominal processing: 366288


In [16]:
merged = merge_children(nominal)

In [17]:
cols = ['sensor',
        'date',
        'L0_fail',
        'L0_success',
        'L1_fail',
        'L1_success',
        'L1_L1G',
        'L1_L1Gt',
        'L1_L1T',
        'nbar',
        'nbart',
        'pq']
result = merged[cols]
result.set_index('date', inplace=True)

In [18]:
# out_fname_fmt = '/g/data/v10/projects/gaip-scripts/harvest-collection/{}-roi.xls'
out_fname_fmt = '/g/data/v10/projects/gaip-scripts/harvest-collection/{}-roi.csv'
collection_stats(result, out_fname_fmt)